In [49]:
#Import necessary modules

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import os
import time
import pandas as pd

In [50]:
#Set Firefox's preferences so that it automatically downloads the PDF 
op = Options()
#Save file to path defined for recent download with value 2
op.set_preference("browser.download.folderList",2)
#Disable display Download Manager window with false value
op.set_preference("browser.download.manager.showWhenStarting", False)
#Make download location the curent working directory
op.set_preference("browser.download.dir",os.getcwd())
#MIME set to save file to disk without asking file type to used to open file
op.set_preference ("browser.helperApps.neverAsk.saveToDisk", "application/pdf;text/plain;application/text;text/xml;application/xml")

#Set permissions to allow automatic download
op.set_preference("browser.download.useDownloadDir", True)

op.set_preference("browser.download.viewableInternally.enabledTypes", "")

op.set_preference("services.sync.prefs.sync.browser.download.manager.showWhenStarting", False)

op.set_preference("pdfjs.disabled", True)

df = pd.DataFrame()

In [51]:
#Open Browser
driver = webdriver.Firefox(options=op)
#Go to the site
driver.get("https://tarjetasube.sube.gob.ar/SubeWeb/Webforms/Account/Views/login.aspx?msg=1")

#Store current window's id
main_page = driver.current_window_handle

#Find and fill credentials
DNI = driver.find_element_by_id("txtDocumento").send_keys("19064900")

#Wait a second to fill the next credential to not alert the captcha
time.sleep(2)

password = driver.find_element_by_id("txtPassword").send_keys("9400")

time.sleep(2)

#Scroll down
driver.execute_script("window.scrollBy(0,500)", "")

time.sleep(3)

#Finds the frame where the captcha is located and switch to it
frame = driver.find_element_by_xpath('//iframe[contains(@src, "recaptcha")]')
driver.switch_to.frame(frame)

#Clicks the captcha and then switches back to the page
driver.find_element_by_xpath("//*[@id='recaptcha-anchor']").click()
driver.switch_to.default_content()

time.sleep(2)

#Clicks to enter
enter = driver.find_element_by_xpath('//button[normalize-space()="Ingresar"]').click()


#Wait until the other page loads. If it waits longer than 10s it closes the page
try:
    elem = WebDriverWait(driver, 10,poll_frequency=3).until(
        EC.presence_of_element_located((By.ID, "dateFrom")) #This is a dummy element
    )
except:
    driver.close()

#Scroll down
driver.execute_script("window.scrollBy(0,500)", "")

time.sleep(1)

#View More
for i in range (10):
    driver.find_element_by_xpath("//*[@class='text-muted ng-binding']").click()
    driver.execute_script("window.scrollBy(0,400)", "")

    
#Get Data    
for table in driver.find_elements_by_xpath('//*[contains(@class,"ng-scope")]//tr'):
    data = [item.text for item in table.find_elements_by_xpath(".//*[self::td or self::th]")]
    row = pd.DataFrame([data])
    df = df.append(row, ignore_index=True)

In [53]:
data = df

#Drop unnecessary rows
data = data.drop([0,1]).reset_index(drop = True)

#Set column names equal to values in row index position 0
data.columns = data.iloc[0]

#Remove first row from DataFrame
data = data[1:]

data

,Fecha y hora,Tipo,Medio,Detalle,Valor
1,24/11/2022 19:21:23,Uso,LINEA D,,"$ -42,00"
2,24/11/2022 17:56:14,Uso,LINEA D,,"$ -42,00"
3,18/11/2022 21:59:41,Uso,LINEA 152,Interno 5,"$ -28,00"
4,18/11/2022 21:54:48,Carga,,Local 68 - Spalla,"$ 100,00"
5,18/11/2022 21:54:13,Carga,,Local 68 - Spalla,"$ 100,00"
...,...,...,...,...,...
90,27/06/2022 07:16:52,Uso con RED SUBE 1,LINEA E,,"$ -15,00"
91,27/06/2022 07:16:38,Carga,LINEA A,Linea A - Acoyte,"$ 200,00"
92,27/06/2022 05:58:10,Uso,LINEA 8,Interno 1122,"$ -40,30"
93,,NaN,NaN,NaN,NaN


In [54]:
# determining the name of the file
file_name = 'SUBE Data.xlsx'
  
# saving the excel
data.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
